# Импорт модулей и библиотек

In [25]:
import pandas as pd
import numpy as np
import pickle
import datetime as d
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from implicit.nearest_neighbours import CosineRecommender, BM25Recommender, TFIDFRecommender
from implicit.als import AlternatingLeastSquares
from implicit.lmf import LogisticMatrixFactorization
from implicit.bpr import BayesianPersonalizedRanking


import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

In [79]:
def extraction_data(path_new_data: str, all_years_df = pd.DataFrame([])):

    # load data 2015-2021 years
    for year in range(2015, 2022):
        with open(f"../Data/df_{year}_cleaned.pickle", "rb") as file:
            df_years = pickle.load(file)
        all_years_df = all_years_df.append(df_years, ignore_index=True)

    all_years_df.rename(columns={"deals": "amount"}, inplace=True)
    all_years_df.drop("date", axis=1, inplace=True)
    
    # load data 2022 years
    df_2022 = pd.read_csv(f"{path_new_data}", sep=";")

    # Some transforms for data 
    df_2022["datetime"] = pd.to_datetime(df_2022["timestamp"])
    df_2022["cf"] = -df_2022["amount"]*df_2022["price"]
    df_2022.rename(columns={"id": "user"}, inplace=True)

    df_2022 = df_2022.reindex(columns=["datetime", "ticker", "amount", "price", "user", "cf"])

    df_result = pd.concat([df_2022, all_years_df]).reset_index(drop = True)
    df_result["volume"] = abs(df_result["cf"])
    
    tick2ind = {ticker:i for i, ticker in enumerate(df_result.ticker.unique())}
    ind2tick = {val:key for key, val in tick2ind.items()}
    df_result["ticker_id"] = df_result["ticker"].apply(lambda x: tick2ind[x])

    user2ind = {user:i for i, user in enumerate(df_result.user.unique())}
    ind2user = {val:key for key, val in user2ind.items()}
    df_result["user_id"] = df_result["user"].apply(lambda x: user2ind[x])
    df_result.groupby(["ticker_id", "user_id", "datetime"], as_index=False)["volume"].sum()
    df_result["year"] = df_result["datetime"].apply(lambda row: row.year)
    
    df_result.sort_values("datetime", inplace = True)
    return df_result.reset_index(drop = True)

In [80]:
df_result = extraction_data("../Data/stock_market_trades.csv")

In [102]:
#df_result = pd.read_csv("../Data/Result_data/resultData.csv", index_col=0)
tick2ind = {ticker:i for i, ticker in enumerate(df_result.ticker.unique())}
ind2tick = {val:key for key, val in tick2ind.items()}
user2ind = {user:i for i, user in enumerate(df_result["user"].unique())}
ind2user = {val:key for key, val in user2ind.items()}

In [103]:
user2ind[311716]

41383

In [83]:
df_result.head(5)

,datetime,ticker,amount,price,user,cf,volume,ticker_id,user_id,year
0,2015-09-16 10:00:00,LKOH,5,2485.10,1_48198,-12425.5,12425.5,4,8851,2015
1,2015-09-16 10:00:00,SBER,100,75.12,1_48199,-7512.0,7512.0,23,8951,2015
2,2015-09-16 10:00:00,SBER,10,75.25,1_49493,-752.5,752.5,23,3355,2015
3,2015-09-16 10:00:00,LKOH,38,2485.10,1_48198,-94433.8,94433.8,4,8851,2015
4,2015-09-16 10:00:00,SBER,-200,75.23,1_48670,15046.0,15046.0,23,7243,2015


## train-test-split

### Создаем словарь (Дата сделки: сделка)

In [84]:
def date_deals(df, data):    
    
    """
    Функция получает на вход столбец датафрейма с датами сделок
    
    На выходе: словарь с ключами - Дата сделки, а значения - сами сделки
    """
    
    date_deal = {}
    dates=set(data.values)
    
    for d in dates:
        deals = df[(data==d)]
        if len(deals)==1:
            continue
        else:
            date_deal[d] = deals    
            
    return date_deal

# Train_test_split

In [165]:
def train_test_split(df, train_size=0.7, test_size=0.3):
    
    train = pd.DataFrame()
    test = pd.DataFrame()
    dict_date = date_deals(df, df["year"])
    
    for key in dict_date.keys():
        deals_count = len(dict_date[key])
        to_train = round(deals_count*train_size)
        to_test = round(deals_count*test_size)
            
        train = pd.concat([train, dict_date[key][:to_train]])
        test = pd.concat([test, dict_date[key][-to_test:]])
    return (train, test)

In [166]:
%%time
train, test = train_test_split(df_result, train_size=0.7, test_size=0.3)

CPU times: total: 3.45 s
Wall time: 3.48 s


In [167]:
test.shape

(2055579, 10)

In [157]:
list_users_train = list(train["user_id"])
for i in list_users_train:
    if i not in list(test["user_id"]):
        test.drop(test[test["user_id"]==i].index, inplace=True)

KeyboardInterrupt: 

In [89]:
# train.to_csv("../Data/Result_data/train.csv")
# test.to_csv("../Data/Result_data/test.csv")

# Сбор матриц

In [91]:
# train = pd.read_csv("../Data/Result_data/train.csv", index_col=0)
# test = pd.read_csv("../Data/Result_data/test.csv", index_col=0)

In [92]:
def get_csr_matrix(df, user_col, ticker_col, add_col, csr_method):
    n_rows = df[user_col].nunique()
    n_cols = df[ticker_col].nunique()

    row = df[user_col] #user
    col = df[ticker_col] #ticker
    data = df[add_col].astype(float) #deals
    
    return csr_method((data, (row, col)))

In [93]:
csr_train = get_csr_matrix(train, "user_id", "ticker_id", "volume", csr_matrix)
csr_test = get_csr_matrix(test, "user_id", "ticker_id", "volume", csr_matrix)

# Обучение

Параметр K влияет на максимальную выдачу топа, поэтому N для таких моделей желательно указывать меньше K

In [95]:
def recommend_tick(user, model, train_matrix, N):
    user_id = user2ind[user]

    recs = model.recommend(user_id, train_matrix[user_id], N=N, filter_already_liked_items=True)
    
    res = [ind2tick[ticker_ids] for ticker_ids in recs[0]]
    return res

In [96]:
def find_similar(user, model, train_matrix, N):
    user_id = user2ind[user]

    recs = model.recommend(user_id, train_matrix[user_id], N=N, filter_already_liked_items=True)
    
    res = recs[1]
    return res

In [179]:
def get_recommenders(model, csr_matrix_train, csr_matrix_test, df_test, 
                     k=20, factors=40, regularization=0.1, iterations=20,
                     top_N = 10):
    
    if model == CosineRecommender or model == BM25Recommender or model == TFIDFRecommender
        fitting_model = model(K=k)
    elif model == AlternatingLeastSquares or model == LogisticMatrixFactorization or model == BayesianPersonalizedRanking:
        fitting_model = model(factors=factors, regularization=regularization, iterations=iterations)
    
    fitting_model.fit(csr_matrix_train)

    df_reccomeds = pd.DataFrame({'user': df_test['user'].unique()})
    
    df_reccomeds['ticker'] = df_reccomeds['user'].apply(lambda x: 
                                                            recommend_tick(x, 
                                                                           fitting_model, 
                                                                           csr_matrix_train, 
                                                                           top_N))
    df_reccomeds['similar'] = df_reccomeds['user'].apply(lambda x: find_similar(x, 
                                                                                fitting_model, 
                                                                                csr_matrix_train, 
                                                                                top_N))
    
    return df_reccomeds

In [174]:
models = {"cosine": CosineRecommender,
          "bm25": BM25Recommender, 
          "tfidf": TFIDFRecommender, 
          "als": AlternatingLeastSquares,
          "lmf": LogisticMatrixFactorization,
          "bpr": BayesianPersonalizedRanking}

In [133]:
dict_df_recommends = dict() 
for key, val in models.items():
    if val == CosineRecommender
    for i in k:
        
        dict_df_recommends[key + k] = get_recommenders(val, csr_train, csr_test, test).explode(["similar", "ticker"])
    dict_df_recommends[key]["rank"] = dict_df_recommends[key].groupby('user')["similar"].cumcount() + 1

  0%|          | 0/161 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

# Метрики

In [198]:
def MRR(test_data, predict_data):

    '''
    RR - обратный ранг: 1 / индеск первого вхождения рекомендации в тест

    На вход подается:
        - test_data - тестовый набор данных
        - predict_data - предсказания

    k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

    На выходе:
        Метрика MRR@k по всем пользователям
    '''

    sum_rr = 0
    # находим порядковый номер первого вхождения правильной рекомендации в тест
    for user in test_data.index.unique():
        rank = []
        if user in predict_data.index:

            rank = [1 + list(predict_data['ticker'][user]).index(x) for x in \
                    set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user]))]
            
            # находим обратный ранк, если попадания нет, то зануляем
            if len(rank) == 0:
                rr = 0
            else:
                rr = 1 / min(rank)

            sum_rr += rr
    

    # находим среднее по пользователям
    return sum_rr / len(test_data.index.unique())

In [190]:
def mean_average_precision_at_k(test_data, predict_data):

    '''
    Mean Average Precision - средняя точность по пользователям

    На вход:
        - test_data - тестовый набор данных
        - predict_data - предсказания

    k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

    На выходе:
        Метрика MAP@k по всем пользователям
    '''

    sum_ap = 0

    # итерируемся по пользователям
    for user in test_data.index.unique():
        
        if user in predict_data.index:
            num_hits = 0
            score = 0

            for i, p in enumerate(predict_data['ticker'][user]):

                if p in test_data['ticker'][user] and p not in predict_data['ticker'][user][:i]:
                    num_hits += 1
                    score += num_hits / (i + 1)

            sum_ap += score / min(len(test_data['ticker'][user]), len(predict_data['ticker'][user]))

    # находим среднее по пользователям
    return sum_ap / len(test_data.index.unique())

In [191]:
def mean_recall(test_data, predict_data):

    '''
     На вход подается:
         - test_data - тестовый набор данных
         - predict_data - предсказания

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     На выходе:
         Средняя метрика recall@k по всем пользователям
    '''

    sum_recall = 0

    # вычисляем метрику по каждому пользователю
    for user in test_data.index.unique():
        if user in predict_data.index:

            TP = len(set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user])))
            TP_FN = len(test_data['ticker'][user])

            sum_recall += TP / TP_FN

    # находим среднее по пользователям
    return sum_recall / len(test_data.index.unique())

In [192]:
def NDCG(test_data, predict_data):
    
    '''
    NDCG - метрика, равная сумме выражений вида 1/log2(k_t+1), деленная на 1/log2(k_all+1) по всем пользователям,
        где k_t - ранг правильно предсказанных акций, k_all - все ранги
    
    
    На вход подается: 
        - test_data - тестовый набор данных 
        - predict_data - предсказания модели
    
    k берется равным длине рассчитанных рекомендаций 
    
    На выходе: 
        Метрика NDCG по всем пользователям
    '''
    
    dcg = 0
    idcg = 0
    
    for user in test_data.index.unique():
        if user in predict_data.index:
            
            # определяем ранги правильно предсказанных акций
            rec_ranks = [1 + list(predict_data['ticker'][user]).index(x) for x in \
                    set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user]))]
            
            dcg += sum([ 1 / log2(r+1) for r in rec_ranks]) # сумма по правильно предсказанным акциям 
            idcg += sum([ 1 / log2(r+1) for r in range(1, 11)]) # максимально возможная сумма
        print(dcg, idcg)
                
    ndcg = dcg/idcg
    
    return ndcglen / (test_data.index.unique())

In [193]:
def calculate_metrics(test_data, predict_data, metrics=[]):
    
    '''
    На вход:
        test_data - тестовый набор данных
        predict_data - рекомендации для пользователя
        metrics - список нужных метрик 
    
    На выходе:
        Вычисленная таблица с результатами 
    '''
    
    if len(metrics) == 0:
        raise ValueError('metrics are empty')
        
    results = []
    for metric in metrics:
        results.append(metric(test_data, predict_data))
    
    return results

In [200]:
models = {"MRR": MRR,
          "MAP": mean_average_precision_at_k, 
          "MAR": mean_recall, 
          "NDCG": NDCG}

In [199]:
print(MRR(test, dict_df_recommends["cosine"]))
# print(mean_recall(test, dict_df_recommends["cosine"]))
# print(mean_average_precision_at_k(test, dict_df_recommends["cosine"]))
# print(NDCG(test, dict_df_recommends["cosine"]))

0.0


In [350]:
%%time
test.index = test['user'].apply(lambda x: user2ind[x])

models = {"cosine": CosineRecommender,
          "bm25": BM25Recommender, 
          "tfidf": TFIDFRecommender, 
          "als": AlternatingLeastSquares,
          "lmf": LogisticMatrixFactorization,
          "bpr": BayesianPersonalizedRanking}

metrics = [MRR, mean_recall, mean_average_precision_at_k, NDCG]

for m in metrics:
    print(calculate_metrics(test, df_reccom, [m]))

#print(pd.DataFrame(calculate_metrics(test, df_reccom, metrics), index=[0]))

{<function MRR at 0x000001D61B111EE0>: 0.0}
{<function mean_recall at 0x000001D60EAD2040>: 0.02508150316830978}


KeyboardInterrupt: 

# Мусорка

In [339]:
test.index = test['user'].apply(lambda x: user2ind[x])

In [32]:
from sklearn.metrics import ndcg_score
#ndcg = ndcg_score(test, df_reccom)

In [27]:
df_reccom = df_reccom.explode(["ticker", "similar"])
df_reccom["rank"] = range(df_reccom.shape[0])
df_reccom["rank"] = df_reccom["rank"].apply(lambda x: x%10+1)

In [46]:
MRR(test.reset_index().drop(["index"], 1), df_reccom)
rank = [1 + list(predict_data['ticker'][user]).index(x) for x in \
        set(predict_data['ticker'][user]).intersection(set(test_data['ticker'][user]))]

[1 + list(df_reccom['ticker'][36]).index(x) for x in \
                    set(df_reccom['ticker'][36]).intersection(set(test['ticker'][36]))]

In [342]:
#(set(test['ticker'].filter(test.user_id == 4)))
for i in range(0, 100):
    s = set(df_reccom['ticker'][i]).intersection(test.loc[test.user_id == i]["ticker"])
    if len(s)!=0:
        print(i, s)

1 {'VTBR', 'ALRS', 'MGNT'}
6 {'PHOR'}
11 {'ALRS', 'RTKM'}
19 {'UPRO', 'ALRS', 'MGNT'}
21 {'GAZP', 'PLZL'}
22 {'PHOR'}
26 {'FIVE'}
27 {'MGNT'}
28 {'PLZL'}
29 {'PLZL', 'ALRS'}
31 {'PHOR', 'SIBN'}
36 {'PLZL', 'DSKY', 'CHMF', 'ALRS'}
38 {'NVTK'}
48 {'PHOR', 'SIBN', 'CHMF', 'PLZL'}
55 {'ALRS'}
60 {'PLZL'}
64 {'SBER', 'LKOH', 'GMKN'}
66 {'AFKS'}
73 {'OGKB'}
75 {'NVTK'}
76 {'PHOR'}
77 {'PLZL', 'NVTK'}
80 {'NVTK'}
83 {'UPRO', 'PLZL'}
89 {'MGNT'}
95 {'MTSS', 'PIKK', 'CHMF', 'NLMK'}
97 {'SIBN'}


In [341]:
set(df_reccom['ticker'][36]).intersection(set(test['ticker'][36]))

set()

In [338]:
set(test['ticker'][36])

{'B', 'E', 'P', 'R', 'S'}

In [334]:
df_reccom

,user,ticker,similar,rank
0,1_95110,ALRS,1.126622,1.0
0,1_95110,FEES,1.071905,2.0
0,1_95110,AKMM,1.067964,3.0
0,1_95110,SBCS,1.054147,4.0
0,1_95110,PLZL,1.053041,5.0
...,...,...,...,...
30747,1_61427,PLZL,0.833334,6.0
30747,1_61427,FXRU,0.832413,7.0
30747,1_61427,RTKM,0.83234,8.0
30747,1_61427,MTLRP,0.81852,9.0


In [340]:
test

,datetime,ticker,amount,price,user,cf,volume,ticker_id,user_id,year
user,,,,,,,,,,
10729,2016-11-29 11:04:00,SBER,10,160.000,1_95110,-1600.0,1600.0,23,12969,2016
11861,2016-11-29 11:04:00,GAZP,50,149.460,1_97230,-7473.0,7473.0,15,14006,2016
10729,2016-11-29 11:04:00,SBER,10,160.000,1_95110,-1600.0,1600.0,23,12969,2016
11861,2016-11-29 11:04:00,GAZP,20,149.460,1_97230,-2989.2,2989.2,15,14006,2016
11861,2016-11-29 11:04:00,GAZP,10,149.460,1_97230,-1494.6,1494.6,15,14006,2016
...,...,...,...,...,...,...,...,...,...,...
2881,2015-12-15 18:49:00,SBERP,100,71.400,1_58056,-7140.0,7140.0,74,6665,2015
3508,2015-12-15 18:49:00,LKOH,10,2446.800,1_58980,-24468.0,24468.0,4,4720,2015
3105,2015-12-15 18:49:00,LKOH,39,2446.800,1_58336,-95425.2,95425.2,4,7156,2015


In [237]:
test = test.reset_index().drop(["index"], 1)
test.index = test['user'].apply(lambda x: user2ind[x])
test

ValueError: cannot insert user, already exists

In [223]:
df_reccom.index

Int64Index([    0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,
            ...
            30747, 30747, 30747, 30747, 30747, 30747, 30747, 30747, 30747,
            30747],
           dtype='int64', length=307480)

In [162]:
rank = [1 + list(df_reccom['ticker'][df_reccom.user_id == 6]).index(x) for x in \
        set(df_reccom['ticker'][df_reccom.user_id == 6]).intersection(set(test['ticker'][df_reccom.user_id == 6]))]

In [325]:
%%time
print(NDCG(test, df_reccom))

0 4.543559338088346
0 9.087118676176692
0 13.630678014265037
0 18.174237352353384
0 22.71779669044173
0 27.261356028530077
0 31.804915366618424
0 36.34847470470677
0 40.89203404279511
0 45.435593380883454
0 49.9791527189718
0 54.52271205706014
0 59.06627139514848
0 63.60983073323683
0 68.15339007132518
0 72.69694940941352
0 77.24050874750186
0 81.7840680855902
0 86.32762742367855
0 90.8711867617669
0 95.41474609985524
0 99.95830543794358
0 104.50186477603192
0 109.04542411412027
0 113.58898345220861
0 118.13254279029695
0 122.6761021283853
0 127.21966146647364
0 131.763220804562
0 136.30678014265035
0 140.8503394807387
0 145.39389881882707
0 149.93745815691543
0 154.48101749500378
0 159.02457683309214
0 163.5681361711805
0 168.11169550926886
0 172.6552548473572
0 177.19881418544557
0 181.74237352353393
0 186.28593286162229
0 190.82949219971064
0 195.373051537799
0 199.91661087588736
0 204.46017021397572
0 209.00372955206407
0 213.54728889015243
0 218.0908482282408
0 222.63440756632914


0 5697.623409962705
0 5702.166969300793
0 5706.710528638881
0 5711.2540879769695
0 5715.7976473150575
0 5720.3412066531455
0 5724.8847659912335
0 5729.4283253293215
0 5733.9718846674095
0 5738.515444005498
0 5743.059003343586
0 5747.602562681674
0 5752.146122019762
0 5756.68968135785
0 5761.233240695938
0 5765.776800034026
0 5770.320359372114
0 5774.863918710202
0 5779.40747804829
0 5783.951037386378
0 5788.494596724466
0 5793.038156062554
0 5797.581715400642
0 5802.12527473873
0 5806.668834076818
0 5811.212393414906
0 5815.755952752994
0 5820.299512091082
0 5824.84307142917
0 5829.386630767258
0 5833.930190105346
0 5838.473749443434
0 5843.017308781522
0 5847.56086811961
0 5852.104427457698
0 5856.647986795786
0 5861.191546133874
0 5865.735105471962
0 5870.27866481005
0 5874.822224148138
0 5879.365783486226
0 5883.909342824314
0 5888.452902162402
0 5892.99646150049
0 5897.540020838578
0 5902.083580176666
0 5906.627139514754
0 5911.170698852842
0 5915.71425819093
0 5920.257817529018
0 

0 11872.32055042432
0 11876.864109762408
0 11881.407669100496
0 11885.951228438584
0 11890.494787776672
0 11895.03834711476
0 11899.581906452848
0 11904.125465790936
0 11908.669025129024
0 11913.212584467112
0 11917.7561438052
0 11922.299703143288
0 11926.843262481376
0 11931.386821819464
0 11935.930381157552
0 11940.47394049564
0 11945.017499833728
0 11949.561059171816
0 11954.104618509904
0 11958.648177847992
0 11963.19173718608
0 11967.735296524168
0 11972.278855862256
0 11976.822415200344
0 11981.365974538432
0 11985.90953387652
0 11990.453093214608
0 11994.996652552696
0 11999.540211890784
0 12004.083771228872
0 12008.62733056696
0 12013.170889905048
0 12017.714449243136
0 12022.258008581224
0 12026.801567919312
0 12031.3451272574
0 12035.888686595488
0 12040.432245933576
0 12044.975805271664
0 12049.519364609752
0 12054.06292394784
0 12058.606483285928
0 12063.150042624016
0 12067.693601962104
0 12072.237161300192
0 12076.78072063828
0 12081.324279976368
0 12085.867839314456
0 12

0 17710.794299867954
0 17715.337859206044
0 17719.881418544133
0 17724.424977882223
0 17728.968537220313
0 17733.512096558403
0 17738.055655896493
0 17742.599215234583
0 17747.142774572672
0 17751.686333910762
0 17756.229893248852
0 17760.773452586942
0 17765.31701192503
0 17769.86057126312
0 17774.40413060121
0 17778.9476899393
0 17783.49124927739
0 17788.03480861548
0 17792.57836795357
0 17797.12192729166
0 17801.66548662975
0 17806.20904596784
0 17810.75260530593
0 17815.29616464402
0 17819.83972398211
0 17824.3832833202
0 17828.92684265829
0 17833.47040199638
0 17838.01396133447
0 17842.55752067256
0 17847.10108001065
0 17851.64463934874
0 17856.18819868683
0 17860.73175802492
0 17865.275317363008
0 17869.818876701098
0 17874.362436039188
0 17878.905995377278
0 17883.449554715367
0 17887.993114053457
0 17892.536673391547
0 17897.080232729637
0 17901.623792067727
0 17906.167351405817
0 17910.710910743906
0 17915.254470081996
0 17919.798029420086
0 17924.341588758176
0 17928.88514809

0 24362.565170831473
0 24367.108730169562
0 24371.652289507652
0 24376.195848845742
0 24380.739408183832
0 24385.28296752192
0 24389.82652686001
0 24394.3700861981
0 24398.91364553619
0 24403.45720487428
0 24408.00076421237
0 24412.54432355046
0 24417.08788288855
0 24421.63144222664
0 24426.17500156473
0 24430.71856090282
0 24435.26212024091
0 24439.805679579
0 24444.34923891709
0 24448.89279825518
0 24453.43635759327
0 24457.97991693136
0 24462.52347626945
0 24467.06703560754
0 24471.61059494563
0 24476.15415428372
0 24480.69771362181
0 24485.2412729599
0 24489.784832297988
0 24494.328391636078
0 24498.871950974168
0 24503.415510312258
0 24507.959069650347
0 24512.502628988437
0 24517.046188326527
0 24521.589747664617
0 24526.133307002707
0 24530.676866340797
0 24535.220425678886
0 24539.763985016976
0 24544.307544355066
0 24548.851103693156
0 24553.394663031246
0 24557.938222369336
0 24562.481781707425
0 24567.025341045515
0 24571.568900383605
0 24576.112459721695
0 24580.65601905978

...

0 135220.86946077377
0 135225.41302011185
0 135229.95657944994
0 135234.50013878802
0 135239.0436981261
0 135243.58725746418
0 135248.13081680227
0 135252.67437614035
0 135257.21793547843
0 135261.7614948165
0 135266.3050541546
0 135270.84861349268
0 135275.39217283076
0 135279.93573216884
0 135284.47929150693
0 135289.022850845
0 135293.5664101831
0 135298.10996952117
0 135302.65352885926
0 135307.19708819734
0 135311.74064753542
0 135316.2842068735
0 135320.8277662116
0 135325.37132554967
0 135329.91488488775
0 135334.45844422583
0 135339.00200356392
0 135343.545562902
0 135348.08912224008
0 135352.63268157816
0 135357.17624091625
0 135361.71980025433
0 135366.2633595924
0 135370.8069189305
0 135375.35047826858
0 135379.89403760666
0 135384.43759694474
0 135388.98115628283
0 135393.5247156209
0 135398.068274959
0 135402.61183429707
0 135407.15539363516
0 135411.69895297324
0 135416.24251231132
0 135420.7860716494
0 135425.3296309875
0 135429.87319032557
0 135434.41674966365
0 135438.

0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.3625

0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.3625

0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.36252746126
0 139705.3625

KeyboardInterrupt: 

In [202]:
k = [1, 2, 3]
b = [4, 5, 6]
c = [7, 8, 9]

for (i, d, f) in zip(k, b, c):
    print(i, d, f)

1 4 7
2 5 8
3 6 9
